In [1]:
from datasets import load_dataset
ds = load_dataset("zh-plus/tiny-imagenet")

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import time

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

print("Libraries imported - ready to use PyTorch", torch.__version__)

def show_image(image, label):
    image = image.permute(1, 2, 0)
    plt.imshow(image.squeeze())
    plt.title(f'Label: {label}')
    plt.show()

Libraries imported - ready to use PyTorch 2.3.1


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from datasets import load_dataset  # Assuming load_dataset loads your dataset

# Define a custom dataset class
class TinyImageNetDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        try:
            image_path = self.dataset[idx]['image']
            image = Image.open(image_path).convert('RGB')
            label = self.dataset[idx]['label']
            filename = str(self.dataset[idx]['filename'])  # Convert Path to string

            if self.transform:
                image = self.transform(image)
            
            if image is None:
                raise ValueError(f"Transformed image is None at index {idx}")

            return {'image': image, 'label': label, 'filename': filename}
        
        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
            return None

# Define the transformation without normalization
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
])

# Load the Tiny ImageNet dataset
ds = load_dataset("zh-plus/tiny-imagenet")

# Create the dataset
dataset = TinyImageNetDataset(ds['train'], transform=transform)

# Create a DataLoader with tqdm for progress bar
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)
dataloader = tqdm(dataloader, desc='Data Loading Progress')  # Wrap with tqdm for progress bar

# Function to compute mean and std
def compute_mean_std(dataloader):
    mean = 0.0
    std = 0.0
    count = 0
    for batch in dataloader:
        images = batch['image']
        
        if images is None:
            continue
        
        batch_samples = images.size(0)  # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2xwx).sum(0)
        std += images.std(2).sum(0)
        count += batch_samples
    
    mean /= count
    std /= count
    return mean, std

# Compute mean and std
mean, std = compute_mean_std(dataloader)
print(f'Mean: {mean}')
print(f'Std: {std}')


Data Loading Progress:   0%|          | 0/3125 [00:00<?, ?it/s]

Error loading image at index 0: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 1: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 2: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 3: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 4: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 5: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 6: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 7: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 8: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 9: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 10: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 11: 'JpegImageFile' object has no attribute 'read'
Error loading image at index 12: 'JpegImageFile' o

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>